In [1]:
from databaker.framework import *

In [9]:
inputfile = "referencetablesjune210815_tcm77-413962.xls"
outputfile = "reftarea.csv"
previewfile = "preview.html"

#bake --preview OTTArea.py %Excel_Sheet%
#bake --preview OTTPurpose.py %Excel_Sheet%
#bake --preview OTTEarnings.py %Excel_Sheet%
tabs = loadxlstabs(inputfile, "*")

dtabs = dict((tab.name, tab)  for tab in tabs)

Loading referencetablesjune210815_tcm77-413962.xls which has size 193536 bytes
Table names: Index, Table 1, Table 2, Table 3, Table 4, Table 5


In [28]:
import re

def monthdim(monthset):
    mcvo = dict((m._cell, re.match("([A-Za-z]*)", m.value).group(1))  for m in monthset)
    return HDim(monthset, "Month", DIRECTLY, LEFT, cellvalueoverride=mcvo)
    
def tabarea(tab):
    anchor = tab.excel_ref("A6")
    req_cols = anchor.fill(RIGHT).is_not_blank()
    req_rows = anchor.fill(DOWN) - tab.filter(contains_string("Latest three months")).expand(DOWN)
    obs = req_rows.waffle(req_cols).is_not_blank()
    dimensions = [ 
        HDim(tab.excel_ref("B1"), "Type", CLOSEST, ABOVE), 
        #req_cols.dimension("SA / NSA", DIRECTLY, ABOVE)
        HDim(tab.excel_ref("3:5").is_not_blank().children(), "Area", DIRECTLY, ABOVE), 
        HDim(req_rows.is_not_blank(), "Year", CLOSEST, ABOVE), 
        monthdim(anchor.shift(RIGHT).fill(DOWN))
    ]
    obs = obs - tab.excel_ref('E1').fill(DOWN)
    return ConversionSegment(tab, dimensions, obs)

def tabearnings(tab):
    assert tab.name == "Table 5"
    anchor = tab.excel_ref("A7")
    req_cols = anchor.fill(RIGHT).is_not_blank()
    req_rows = anchor.fill(DOWN) - tab.filter(contains_string("Latest three months")).expand(DOWN)
    obs = req_rows.waffle(req_cols).is_not_blank()
    dimensions = [ 
        HDim(tab.excel_ref("B1"), "Type", CLOSEST, ABOVE),  
        HDim(req_cols, "SA / NSA", DIRECTLY, ABOVE), 
        HDim(tab.excel_ref("A5").expand(RIGHT).parent(), "Area", CLOSEST, LEFT),
        HDim(req_rows.is_not_blank(), "Year", CLOSEST, ABOVE), 
        monthdim(anchor.shift(RIGHT).fill(DOWN))
    ]
    obs = obs - tab.excel_ref('J1').expand(DOWN).expand(RIGHT)    
    return ConversionSegment(tab, dimensions, obs)
 
def tabpurpose(tab):
    anchor = tab.excel_ref("A6")
    req_cols = anchor.fill(RIGHT).is_not_blank()
    req_rows = anchor.fill(DOWN) - tab.filter(contains_string("Latest three months")).expand(DOWN)
    obs = req_rows.waffle(req_cols).is_not_blank()
    dimensions = [
        HDim(tab.excel_ref("B1"), "Type", CLOSEST, ABOVE), 
        # req_cols.dimension("SA / NSA", DIRECTLY, ABOVE)
        HDim(tab.excel_ref("3:4").is_not_blank().children(), "Purpose", DIRECTLY, ABOVE), 
        HDim(req_rows.is_not_blank(), "Year", CLOSEST, ABOVE), 
        monthdim(anchor.shift(RIGHT).fill(DOWN))
    ]
    obs = obs - tab.excel_ref('E1').fill(DOWN) 
    return ConversionSegment(tab, dimensions, obs)
    
#cs1 = tabarea(dtabs["Table 1"])
cs1 = tabpurpose(dtabs["Table 2"])
savepreviewhtml(cs1, previewfile)        
df = topandas(cs1)
conversionsegments = [ tabarea(dtabs["Table 1"]), tabarea(dtabs["Table 3"]), tabearnings(dtabs["Table 5"]), 
                       tabpurpose(dtabs["Table 2"]), tabpurpose(dtabs["Table 4"]) ]


tablepart 'Table 2' written #injblock1019
javascript calculated


In [29]:
writetechnicalCSV(outputfile, conversionsegments)

writing 5 conversion segments into /home/goatchurch/sensiblecode/quickcode-ons-recipes/OTT/reftarea.csv
conversionwrite segment size 294 table Table 1
conversionwrite segment size 294 table Table 3
conversionwrite segment size 168 table Table 5
conversionwrite segment size 210 table Table 2
conversionwrite segment size 210 table Table 4
